In [41]:
from openai import OpenAI
import spacy
spacy.prefer_gpu()
spacy_model = "en_core_web_lg"
nlp = spacy.load(spacy_model)



# use your own key
api_key = ""
client = OpenAI(api_key=api_key)
model_id = "gpt-4-1106-preview"

# $0.01 / 1K tokens
if model_id == "gpt-4-1106-preview":
    word_limit = 128000
    
# $0.0010 / 1K tokens
elif model_id == "gpt-3.5-turbo-1106":
    word_limit = 16385

    

file_path = r"C:\Users\ee527\Desktop\Big Data Analytics\final_project\podcast_transcript\3\The Full Indictment Read by Ali Velshi.txt"
with open(file_path, 'r') as file:
    file_contents = file.read()
    
print("Using OpenAI model: ", model_id)
print("Using spaCy model: ", spacy_model)

Using OpenAI model:  gpt-4-1106-preview
Using spaCy model:  en_core_web_lg


### Pre-processing

In [42]:
#sentences = file_contents.split(".")
word_length = len(file_contents.split())
print("Total Words in the transcript: ", word_length)


doc = nlp(file_contents)
sentences = []
for sentence in doc.sents:
    sentences.append(sentence.text)
    
print("Total sentences in the transcript: ", len(sentences))


print(f"{model_id} maximun tokens: {word_limit}")
if word_length > word_limit:
    print(f"Exceed maximun tokens!!")

Total Words in the transcript:  10301
Total sentences in the transcript:  587
gpt-4-1106-preview maximun tokens: 128000


In [43]:
print(sentences)

[' Hello and welcome to a special episode of prosecuting Donald Trump.', "I'm Ali Velshi.", 'The Department of Justice has unsealed its 49-page federal indictment, charging former President Donald Trump with 37 felony counts in his handling of classified information.', 'This is an historic and important document, and I want to read it to you so that you got every detail about what exactly he and his aide Walt Notta are being charged with.', "You've heard the highlights about how Trump allegedly lied and schemed to keep documents he knew were classified.", "But I'll read word for word the transcript of the conversation Trump allegedly had about a sensitive military document.", "It's between his employees and the descriptions of the documents he allegedly hid in a shower, his bathroom, and a ballroom.", "Trump of course maintains his innocence, but I'll let you be the judge of that.", 'So here we go.', 'United States of America versus Donald J. Trump and Walt Notta indictment.', 'The gra

### GPT API

In [44]:
# prompt engineer here, need to change here 
number = 10

prompt = f"Step1. Read this transcript below first: \n{file_contents}\nStep2. Now give me {number} questions and the precise reference sentence you find in the transcript. Don't ask same questions.\nRule1. There are some advertisement inside this transcript, you need to recognize them and don't use those information to ask question.\nRule2. You musn't paraphrase and change words, structure in the reference sentence. The reference sentence must be completely same and can find in the transcript.\nRule3. Your response should be this format below: \nQuestion: \nReference Sentence: \n\n"


print(prompt)


messages = [
    
        {"role": "user", "content": prompt}

    ]

Step1. Read this transcript below first: 
 Hello and welcome to a special episode of prosecuting Donald Trump. I'm Ali Velshi. The Department of Justice has unsealed its 49-page federal indictment, charging former President Donald Trump with 37 felony counts in his handling of classified information. This is an historic and important document, and I want to read it to you so that you got every detail about what exactly he and his aide Walt Notta are being charged with. You've heard the highlights about how Trump allegedly lied and schemed to keep documents he knew were classified. But I'll read word for word the transcript of the conversation Trump allegedly had about a sensitive military document. It's between his employees and the descriptions of the documents he allegedly hid in a shower, his bathroom, and a ballroom. Trump of course maintains his innocence, but I'll let you be the judge of that. So here we go. United States of America versus Donald J. Trump and Walt Notta indictmen

In [45]:
# initial gpt and start inference
# dont need to change here
completion = client.chat.completions.create(
  model=model_id,
  messages=messages,
)

response = completion.choices[0].message.content
print(response)

Question: Who is the host of the special episode of prosecuting Donald Trump?
Reference Sentence: Hello and welcome to a special episode of prosecuting Donald Trump. I'm Ali Velshi.

Question: How many felony counts is Donald Trump charged with?
Reference Sentence: The Department of Justice has unsealed its 49-page federal indictment, charging former President Donald Trump with 37 felony counts in his handling of classified information.

Question: What is the name of Donald Trump's aide mentioned in the charges?
Reference Sentence: You've heard the highlights about how Trump allegedly lied and schemed to keep documents he knew were classified. But I'll read word for word the transcript of the conversation Trump allegedly had about a sensitive military document. It's between his employees and the descriptions of the documents he allegedly hid in a shower, his bathroom, and a ballroom. Trump of course maintains his innocence, but I'll let you be the judge of that.

Question: When did Don

### Post-processing

In [47]:
# test question or answer

import string
def remove_punctuation(sentence):
    
    punctuation = string.punctuation
    
    while sentence and sentence[0] in punctuation:
        sentence = sentence[1:]
    
    while sentence and sentence[-1] in punctuation:
        sentence = sentence[:-1]
    
    return sentence

    
lines = [line.strip() for line in response.split("\n") if line.strip()]


questions_and_answers = []
for line in lines:
  

    if "Question:" in line:
            print("q:")
            
            colon_index = line.index(':')

            # Extract the sentence after the first colon
            question = line[colon_index + 1:].strip()
            print(question)
            
            
    elif "Reference Sentence:" in line:
        print("a:")
        
        colon_index = line.index(':')

        # Extract the sentence after the first colon
        answer = line[colon_index + 1:].strip()
        
        # clean response here
        
        # clean head and tail punctuation marks
        answer = remove_punctuation(answer)
        
        # clean "..."
        answer = answer.replace("...", "")
        
        print(answer)
     
        questions_and_answers.append((question, answer))
        
    else:
        continue


q:
Who is the host of the special episode of prosecuting Donald Trump?
a:
Hello and welcome to a special episode of prosecuting Donald Trump. I'm Ali Velshi
q:
How many felony counts is Donald Trump charged with?
a:
The Department of Justice has unsealed its 49-page federal indictment, charging former President Donald Trump with 37 felony counts in his handling of classified information
q:
What is the name of Donald Trump's aide mentioned in the charges?
a:
You've heard the highlights about how Trump allegedly lied and schemed to keep documents he knew were classified. But I'll read word for word the transcript of the conversation Trump allegedly had about a sensitive military document. It's between his employees and the descriptions of the documents he allegedly hid in a shower, his bathroom, and a ballroom. Trump of course maintains his innocence, but I'll let you be the judge of that
q:
When did Donald Trump cease to be president?
a:
Four, at 12 p.m. on January 20, 2021, Trump cease

In [48]:
# sentence matching

for qa in questions_and_answers:
    
        question, answer = qa
        
        print("gpt question: ", question)
        print("gpt response: ", answer)
        
        ans_start = file_contents.find(answer)
        
        # first word use lowercase
        if ans_start == -1:
          
          lowercase_answer = answer[0].lower() + answer[1:]
          ans_start = file_contents.find(lowercase_answer)
          
          
          # determine by sentence similarity (spacy)
          if ans_start == -1:
              
              similarities = []
              
              target_doc = nlp(answer)
              for sentence in sentences:
                  
                  doc = nlp(sentence)
                  similarity = doc.similarity(target_doc)
                  similarities.append(similarity)
              
              max_similarity = max(similarities)
              max_similarity_index = similarities.index(max_similarity)
              
              
              print("similarity: ", max(similarities))
              print("transcript: ", sentences[max_similarity_index])
              
              if max_similarity >= 0.95:
                  
                  ans_start = file_contents.find(sentences[max_similarity_index])
                  
              else:
                  print("Skip this error gpt response")
                  ans_start = -1
              
              
              
          else:
              print("Find! The first word in transcript is lowercase.")
              
        else:
            print("Find! Completely match.")
              
              
        print()


gpt question:  Who is the host of the special episode of prosecuting Donald Trump?
gpt response:  Hello and welcome to a special episode of prosecuting Donald Trump. I'm Ali Velshi
Find! Completely match.

gpt question:  How many felony counts is Donald Trump charged with?
gpt response:  The Department of Justice has unsealed its 49-page federal indictment, charging former President Donald Trump with 37 felony counts in his handling of classified information
Find! Completely match.

gpt question:  What is the name of Donald Trump's aide mentioned in the charges?
gpt response:  You've heard the highlights about how Trump allegedly lied and schemed to keep documents he knew were classified. But I'll read word for word the transcript of the conversation Trump allegedly had about a sensitive military document. It's between his employees and the descriptions of the documents he allegedly hid in a shower, his bathroom, and a ballroom. Trump of course maintains his innocence, but I'll let you

### test spacy

In [ ]:
import spacy
spacy.prefer_gpu()

nlp = spacy.load("en_core_web_lg")

your_sentence = "Well, it's been a truly historic week as a Manhattan Grand jury voted to indict former President Donald Trump."

sentences = [
    "Natural language processing (NLP) is a field of computer science, artificial intelligence.",
    "NLP techniques are used to analyze, understand, and generate human language in a valuable way.",
    "It focuses on the interaction between computers and humans through natural language.",
    "Well, it's been a truly historic week as a Manhattan Grand jury voted to in Diteformer President Donald Trump."
  
]


target_doc = nlp(your_sentence)
for sentence in sentences:
    doc = nlp(sentence)
    similarity = doc.similarity(target_doc)
    print(similarity)

In [ ]:
import spacy
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_lg")

path = r"C:\Users\ee527\Desktop\Big Data Analytics\final_project\podcast_transcript\4\5 million In damages.txt"


with open(path, "r", encoding="utf-8") as file:
    text = file.read()

aaa = text.split(".")
print(len(aaa))

doc = nlp(text)

sentence_list = []
for sentence in doc.sents:
   
    sentence_list.append(sentence.text)
print(len(sentence_list))


In [ ]:
print(aaa)

In [ ]:
print(sentence_list[0])

In [ ]:
print(sentence_list[0])

your_sentence = "Hello and welcome."


target_doc = nlp(your_sentence)
doc = nlp(sentence_list[0])
similarity = doc.similarity(target_doc)
print(similarity)